In [10]:
import pandas as pd
from datetime import timedelta
import glob
import os
import matplotlib.pyplot as plt
import numpy as np

In [11]:
df = pd.read_csv(r"C:\Users\Administrator\Desktop\raphi_other\repositories\ts\data/raw/blood_glucose/train.csv")

C:\Users\Administrator\AppData\Local\Temp\ipykernel_5540\4205522785.py:1: DtypeWarning: Columns (435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r"C:\Users\Administrator\Desktop\raphi_other\repositories\ts\data/raw/blood_glucose/train.csv")


In [53]:
os.getcwd()

'C:\\Users\\Administrator\\Desktop\\raphi_other\\repositories\\ts'

In [55]:
os.chdir('C:\\Users\\Administrator\\Desktop\\raphi_other\\repositories\\ts')

In [57]:
from src.bsugar_helpers import downcast_float64_to_float32_and_objects_to_categories


df = downcast_float64_to_float32_and_objects_to_categories(df)

In [72]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras import regularizers
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from math import sqrt
import os
import joblib 
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from category_encoders import BaseNEncoder as CE_BaseNEncoder
import joblib
import pandas as pd

# Directory to save the trained autoencoders and RandomForest model
AUTOENCODER_SAVE_DIR_AUTO = r"../encoders_auto"
AUTOENCODER_SAVE_DIR_BASE = r"../encoders_base"
MODEL_SAVE_DIR = "../models"
os.makedirs(AUTOENCODER_SAVE_DIR_AUTO, exist_ok=True)
os.makedirs(AUTOENCODER_SAVE_DIR_BASE, exist_ok=True)
os.makedirs(MODEL_SAVE_DIR, exist_ok=True)


# Before the autoencoding loop
columns_activities = ['activity-5:55', 'activity-5:50', 'activity-5:45', 'activity-5:40',
   'activity-5:35', 'activity-5:30', 'activity-5:25', 'activity-5:20',
   'activity-5:15', 'activity-5:10', 'activity-5:05', 'activity-5:00',
   'activity-4:55', 'activity-4:50', 'activity-4:45', 'activity-4:40',
   'activity-4:35', 'activity-4:30', 'activity-4:25', 'activity-4:20',
   'activity-4:15', 'activity-4:10', 'activity-4:05', 'activity-4:00',
   'activity-3:55', 'activity-3:50', 'activity-3:45', 'activity-3:40',
   'activity-3:35', 'activity-3:30', 'activity-3:25', 'activity-3:20',
   'activity-3:15', 'activity-3:10', 'activity-3:05', 'activity-3:00',
   'activity-2:55', 'activity-2:50', 'activity-2:45', 'activity-2:40',
   'activity-2:35', 'activity-2:30', 'activity-2:25', 'activity-2:20',
   'activity-2:15', 'activity-2:10', 'activity-2:05', 'activity-2:00',
   'activity-1:55', 'activity-1:50', 'activity-1:45', 'activity-1:40',
   'activity-1:35', 'activity-1:30', 'activity-1:25', 'activity-1:20',
   'activity-1:15', 'activity-1:10', 'activity-1:05', 'activity-1:00',
   'activity-0:55', 'activity-0:50', 'activity-0:45', 'activity-0:40',
   'activity-0:35', 'activity-0:30', 'activity-0:25', 'activity-0:20',
   'activity-0:15', 'activity-0:10', 'activity-0:05', 'activity-0:00']

columns_to_encode = ['p_num', 'time']

## Start

In [74]:
import category_encoders as ce
from sklearn.base import BaseEstimator, TransformerMixin
import torch
import torch.nn as nn
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
import pandas as pd

def encode_baseN(df, cols, base=3):
    print("base N encoding")
    # Instantiate the encoder
    encoder = ce.BaseNEncoder(cols=cols, return_df=True, base=base)
    # Fit and transform the data
    df_encoded = encoder.fit_transform(df[cols])
    return df_encoded, encoder

class BaseNEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, columns, base=3):
        self.columns = columns
        self.base = base
        self.encoder = None
        self.new_column_names = None

    def fit(self, X, y=None):
        # Use category_encoders.BaseNEncoder
        self.encoder = CE_BaseNEncoder(cols=self.columns, base=self.base)
        self.encoder.fit(X[self.columns], y)
        self.new_column_names = self.encoder.get_feature_names_out()
        return self

    def transform(self, X):
        # Transform using the fitted encoder
        X_encoded = self.encoder.transform(X[self.columns])
        X_encoded.index = X.index  # Align indices
        return X_encoded

    def get_feature_names_out(self, input_features=None):
        return self.new_column_names

    def save_encoder(self, path):
        joblib.dump(self.encoder, path)
        print(f"BaseNEncoder saved to {path}")


class ActivityAutoEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, encoding_dim, autoencoder_id):
        self.encoding_dim = encoding_dim
        self.autoencoder_id = autoencoder_id
        self.autoencoder = None

    def fit(self, X, y=None):
        # Fit the autoencoder on the provided data
        self.autoencoder = autoencode_columns_999(
            X, column_indices=None, encoding_dim=self.encoding_dim, autoencoder_id=self.autoencoder_id
        )
        return self

    def transform(self, X):
        # Transform the data using the fitted autoencoder
        X_autoencoded = autoencode_columns_999(
            X, column_indices=None, encoding_dim=self.encoding_dim, autoencoder_id=self.autoencoder_id, train=False
        )
        X_autoencoded_df = pd.DataFrame(X_autoencoded, index=X.index)
        return X_autoencoded_df




In [76]:
target_col='bg+1:00'
# Drop 'id' and target columns
X = df.drop(columns=['id', target_col])  # Features
y = df[target_col]  # Target

print("Splitting data into train/test sets...")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)
print(f"Train set size: {X_train.shape}, Test set size: {X_test.shape}")

# Check how many columns you have for autoencoding
total_columns = X_train.shape[1]
print(f"Total columns to autoencode: {total_columns}")

column_indices_activities = [435,507]

Splitting data into train/test sets...
Train set size: (141619, 506), Test set size: (35405, 506)
Total columns to autoencode: 506


In [77]:
from sklearn.compose import ColumnTransformer

# Define the column transformer
preprocessor = ColumnTransformer(transformers=[
    # Base-N encode 'p_num' and 'time' columns
    ('baseN_pnum_time', BaseNEncoder(columns=['p_num', 'time'], base=3), ['p_num', 'time']),
    
    # Base-N encode activity columns and then autoencode them
    ('activity_pipeline', Pipeline(steps=[
        ('baseN_encoder', BaseNEncoder(columns=columns_activities, base=3)),
        ('autoencoder', ActivityAutoEncoder(encoding_dim=5, autoencoder_id=999))
    ]), columns_activities),
    
    # Pass through other columns unchanged
    ('passthrough', 'passthrough', list(set(X_train.columns) - set(['p_num', 'time']) - set(columns_activities))),
])

# Create the pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
])

# Fit and transform the pipeline
X_train_processed = pipeline.fit_transform(X_train)
X_test_processed = pipeline.transform(X_test)

# Convert to DataFrame if necessary
X_train_processed_df = pd.DataFrame(X_train_processed, index=X_train.index)
X_test_processed_df = pd.DataFrame(X_test_processed, index=X_test.index)

print("Processed training data shape: ", X_train_processed_df.shape)
print("Processed test data shape: ", X_test_processed_df.shape)


TypeError: 'NoneType' object is not subscriptable

In [ ]:
# Save the pipeline for future use
pipeline_save_path = r'../models/final_pipeline.pkl'
joblib.dump(pipeline, pipeline_save_path)
print(f"Final pipeline saved to {pipeline_save_path}")
